In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from scipy.integrate import solve_ivp

In [ ]:
import astropy.constants as ac

#Constants to be used later
GMsun = ac.GM_sun.value                                  #G*(Solar Mass)
c = ac.c.value                                           #Speed of light in vacuum
dsun = GMsun/(c**2)                                      #Natural length scale
tsun = GMsun/(c**3)                                      #Natural time scale
pc = ac.pc.value                                         #One parsec
yr = (365.25)*(24)*(60)*(60)                             #Seconds in one year

In [ ]:
def domg_dt(omg,Mc,eta):
  tN_omg=Mc*omg*tsun
  nu=1
  PN_term=((59/18*eta**(6/5)+13661/2016*eta**(1/5)+34103/18144/
  eta**(4/5))*tN_omg**(4/3)*nu**2+4/eta**(3/5)*nu**(3/2)*pi*tN_omg+(-11/4*eta**(3/5)
  - 43/336/eta**(2/5))*tN_omg**(2/3)*nu)
  res=(96/5*omg**2*tN_omg**(5/3))#*(1+PN_term)
  return res


In [ ]:
domg_dt(1e-9,1e8,0.25)

In [ ]:
sh = int(1e4)

# Define low and high bounds for each dimension
lows = [0, 0, 1, 10, 0.1]              # Lower bounds: tarr_, phi0_, lomg_0, lM_, eta_
highs = [5, 2 * pi, 10, 20, 0.25]       # Upper bounds: tarr_, phi0_, lomg_0, lM_, eta_

# Generate 5D uniform random samples in one call
samples = np.random.uniform(low=lows, high=highs, size=(sh, 5))

In [ ]:
samples.shape[0]

In [ ]:
tarr=np.linspace(0,5,int(1e3))

In [ ]:
samples[1]

In [ ]:
samples[1][0],samples[1][1],samples[1][2]

In [ ]:
def get_PHI_omg(t0, phi0, omg0, Mc, eta, tarr=None, t_epoch=None):
    Mc = Mc * tsun  # Convert Mc to seconds

    # Define the system of ODEs
    def odes(t, y):
        phi, omg = y
        dphi_dt = omg
        domg_dt_val = domg_dt(omg, Mc, eta)
        return [dphi_dt, domg_dt_val]

    y0 = [phi0, omg0]

    # Handle input cases
    if t_epoch is not None:
        # Evaluate only at t_epoch
        tarr_eval = [t_epoch]
        t_span = (t0, t_epoch)
    elif tarr is not None:
        tarr_eval = tarr
        t_span = (tarr[0], tarr[-1])
    else:
        raise ValueError("You must provide either tarr or t_epoch")

    # Solve the ODE
    solution = solve_ivp(
        fun=odes,
        t_span=t_span,
        y0=y0,
        method='RK45',
        t_eval=tarr_eval,
        rtol=1e-6,
        atol=1e-8
    )

    phi, omg = solution.y
    if t_epoch is not None:
        return phi[0], omg[0]  # Single values at epoch
    else:
        return phi, omg        # Arrays over tarr

In [ ]:
from tqdm import tqdm

In [ ]:

#Defining function for omega(t)
def Omega_ext(t, t0, omega0, Mc, eta):
    '''
    t = time
    t0 = reference epoch
    omega0 = omega(t0)
    M = total mass of the SMBHB
    eta = symmetric mass ratio
    N = 0, 1, 1.5 or 2 (0--> Newtonian order, 1--> Upto 1PN, 1.5--> Upto 1.5PN, 2--> Upto 2PN)
    '''

    #Defining some parameters/scaled variables
    #Mc = (M)*((eta)**(3/5))
    M=Mc/((eta)**(3/5))
    tm_ = tsun*M*omega0
    tau = 1 - (((256)*((Mc)**(5/3))*((tsun)**(5/3))*((omega0)**(8/3))*(t - t0))/5)

    
    return omega0/(tau**(3/8))

def Phi(t, t0, phi0, omega0, Mc, eta):
    '''
    t = time
    t0 = reference epoch
    phi0 = phi(t0)
    omega0 = omega(t0)
    M = total mass of the SMBHB
    eta = symmetric mass ratio
    N = 0, 1, 1.5 or 2 (0--> Newtonian order, 1--> Upto 1PN, 1.5--> Upto 1.5PN, 2--> Upto 2PN)
    '''

    #Defining function for 'indefinite' Phi(omega)
    N=0
    def phi(t, t0, phi0, omega0, Mc, eta):

        #Getting omega(t)
        omg = Omega_ext(t, t0, omega0, Mc, eta)

        #Defining some parameters/scaled variables
        #Mc = (M)*((eta)**(3/5))
        tN_omg = tsun*Mc*omg

        #Getting phi(t)
        
        return - 1 / (32 * tN_omg**(5/3))
    return phi(t, t0, phi0, omega0, Mc, eta) - phi(t0, t0, phi0, omega0, Mc, eta) + phi0

In [ ]:
results = np.zeros((samples.shape[0], 2))  # shape (1_000_000, 2)

for i in tqdm(range(samples.shape[0]), desc="Processing samples"):
    t, phi0_, omg0_, Mc_, eta_ = samples[i]

    # omg_val=Omega_ext(t, 0, omg0_, Mc_, eta_)
    # phi_val=Phi(t, 0, phi0_, omg0_, Mc_, eta_)
    phi_val, omg_val = get_PHI_omg(t0=0, phi0=phi0_, omg0=omg0_, Mc=Mc_, eta=eta_, t_epoch=t)
    results[i, 0] = phi_val
    results[i, 1] = omg_val

In [ ]:
results.shape

In [ ]:
plt.plot(samples[:,0],results[:,0],marker='.',ls='')

In [ ]:
plt.plot(samples[:,0],results[:,1],marker='.',ls='')

In [ ]:
data=np.concatenate([samples, results], axis=1)

In [ ]:
data.shape

In [ ]:
import os
outdir="testing"
os.makedirs(outdir, exist_ok=True)

In [ ]:
import torch
data_tensor = torch.from_numpy(data).float()  # Convert to float32 for compatibility

# Save the tensor to a .pt file
torch.save(data_tensor, f'{outdir}/data.pt')